<center><u><b><h1 style="color:Red;">Titanic : Machine Learning from Disaster</u></b></h1></center>

<b>Loading Data</b>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load data
train_df = pd.read_csv(r"C:\Kaggles\Titanic\Dataset\train.csv")
test_df = pd.read_csv(r"C:\Kaggles\Titanic\Dataset\test.csv")
test_ids = test_df["PassengerId"]

# Combine train and test datasets to handle them together
combine = [train_df, test_df]

# Combine datasets for preprocessing
test_df['Survived'] = np.nan  # Add a dummy 'Survived' column to align columns
combined_df = pd.concat([train_df, test_df])

print(train_df.shape, test_df.shape, combined_df.shape)
combined_df.head()

<b>Removing Unrequired Columns</b>

In [ ]:
combined_df.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1, inplace=True)
combined_df.head()

<b>Handling NaN values</b>

In [ ]:
combined_df['Age'].fillna(combined_df['Age'].median(), inplace=True)
combined_df['Embarked'].fillna(combined_df['Embarked'].mode()[0], inplace=True)
combined_df['Fare'].fillna(combined_df['Fare'].median(), inplace=True)

<b>Feature Engineering</b>

In [ ]:
combined_df['FamilySize'] = combined_df['SibSp'] + combined_df['Parch'] + 1
combined_df['IsAlone'] = 1
combined_df.loc[combined_df['FamilySize'] > 1, 'IsAlone'] = 0

In [ ]:
combined_df.head()

In [ ]:
combined_df.drop(["SibSp", "Parch"], axis=1, inplace=True)

In [ ]:
combined_df.head()

<b>Handling Categorical Features</b>

In [ ]:
# Convert categorical features to numeric
combined_df['Sex'] = combined_df['Sex'].map({'female': 1, 'male': 0}).astype(int)
combined_df['Embarked'] = combined_df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)

<b>Seperating the Data back</b>

In [ ]:
# Separate back into train and test sets
train_df = combined_df[combined_df['Survived'].notna()]
test_df = combined_df[combined_df['Survived'].isna()].drop('Survived', axis=1)

In [ ]:
X_train_full = train_df.drop('Survived', axis=1)
y_train_full = train_df['Survived']
X_test = test_df.copy()

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.12, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Build the model
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')

# Detailed classification report
print("Classification Report:")
print(classification_report(y_val, y_pred))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))


<b>Ouput for test Dataset</b>

In [ ]:
# Make predictions on the test set
predictions = model.predict(X_test).astype(int)

# Create submission file
submission = pd.DataFrame({
    "PassengerId": pd.read_csv(r"C:\Kaggles\Titanic\Dataset\test.csv")["PassengerId"],
    "Survived": predictions
})

submission.to_csv(r"C:\Kaggles\Titanic\Submissions\Submission_2.csv", index=False)